In [1]:
import pandas as pd
import matplotlib
import numpy as np

In [2]:
pd.__version__

'2.1.2'

In [122]:
data_name = "nasdaq_Linear.csv"

In [123]:
data = pd.read_csv("..\..\Data\Day\\" + data_name, index_col = 0)

In [124]:
def data_transformer(data):
    if 'Close' in data.columns:
        data = data.replace(",", "", regex=True)
        data.Date = pd.to_datetime(data.Date)
        data = data.sort_values(by = ["Date"])
        data = data.set_index('Date')
        data.Close = data.Close.astype(float)
        data.High = data.High.astype(float)
        data.Low = data.Low.astype(float)
        data.Open = data.Open.astype(float)
        data["Vol"] = data["Vol"].astype(float)
    else:
        data = data.replace(",", "", regex = True)
        data.Date = pd.to_datetime(data.Date)
        data = data.sort_values(by=["Date"])
        data = data.set_index('Date')
        data.Price = data.Price.astype(float)
        data.High = data.High.astype(float)
        data.Low = data.Low.astype(float)
        data.Open = data.Open.astype(float)
        data["Vol."] = data["Vol."].astype(float)

def to_weeks_transfer(df):
    return df.groupby(df.index.to_period('W')).last()

In [125]:
data_transformer(data)

In [126]:
data.head()

,Date,Open,High,Low,Close,Vol
0,2010-01-04,20.2400,20.25,20.22,20.22,63202.0
1,2010-01-05,20.2902,20.30,20.28,20.28,24691.0
2,2010-01-06,20.2800,20.29,20.27,20.29,12756.0
3,2010-01-07,20.0200,20.03,20.01,20.01,40267.0
4,2010-01-08,20.1900,20.22,20.19,20.22,20207.0


In [127]:
data['Date'] = pd.to_datetime(data['Date'])

In [128]:
# Устанавливаем столбец 'Date' в качестве индекса
data.set_index('Date', inplace=True)

In [129]:
data

,Open,High,Low,Close,Vol
Date,,,,,
2010-01-04,20.2400,20.250,20.220,20.2200,63202.0
2010-01-05,20.2902,20.300,20.280,20.2800,24691.0
2010-01-06,20.2800,20.290,20.270,20.2900,12756.0
2010-01-07,20.0200,20.030,20.010,20.0100,40267.0
2010-01-08,20.1900,20.220,20.190,20.2200,20207.0
...,...,...,...,...,...
2013-12-27,39.8900,39.900,39.890,39.9000,900.0
2013-12-28,39.9150,39.930,39.890,39.9300,19872.0
2013-12-29,39.8975,39.905,39.875,39.8950,10186.0


In [130]:
if "Vol." in data:
    data = data.drop(['Vol.'], axis = 1)
if 'Change %' in data:
    data = data.drop(['Change %'], axis = 1)

In [131]:
# Проверяем, является ли индекс DataFrame типом DatetimeIndex
if not isinstance(data.index, pd.DatetimeIndex):
    raise TypeError("Index must be DatetimeIndex")

In [132]:
# Группируем данные по неделям и вычисляем среднее значение для каждого столбца
weekly_df = data.resample('W').mean()

In [133]:
# Сбрасываем индекс, чтобы столбец с датами снова стал обычным столбцом
weekly_df.reset_index(inplace=True)

In [134]:
weekly_df

,Date,Open,High,Low,Close,Vol
0,2010-01-10,20.210743,20.224286,20.203571,20.213571,30680.071429
1,2010-01-17,20.153333,20.162857,20.144762,20.153571,39808.571429
2,2010-01-24,19.186667,19.189286,19.170952,19.178571,39243.500000
3,2010-01-31,18.192829,18.202857,18.182143,18.188571,44122.428571
4,2010-02-07,18.618571,18.632143,18.607857,18.626457,43782.071429
...,...,...,...,...,...,...
204,2013-12-08,39.166429,39.175714,39.162857,39.167857,17596.214286
205,2013-12-15,38.770357,38.771786,38.761286,38.763929,8649.142857
206,2013-12-22,39.520714,39.530000,39.509300,39.514286,25619.785714
207,2013-12-29,39.903929,39.910714,39.895714,39.907857,4858.428571


In [135]:
start_date = weekly_df['Date'].iloc[0]

# Создаем новый столбец 'Day' со значениями дней, начиная с 0
weekly_df['Day'] = pd.Series(range(len(data)))

# Создаем новый столбец 'New_Date' с датами, начиная с первого значения даты
weekly_df['New_Date'] = start_date + pd.to_timedelta(weekly_df['Day'], unit='D')

# Заменяем столбец 'Date' столбцом 'New_Date'
weekly_df['Date'] = weekly_df['New_Date']

# Удаляем временные столбцы 'Day' и 'New_Date'
df = weekly_df.drop(['Day', 'New_Date'], axis=1)

In [136]:
df

,Date,Open,High,Low,Close,Vol
0,2010-01-10,20.210743,20.224286,20.203571,20.213571,30680.071429
1,2010-01-11,20.153333,20.162857,20.144762,20.153571,39808.571429
2,2010-01-12,19.186667,19.189286,19.170952,19.178571,39243.500000
3,2010-01-13,18.192829,18.202857,18.182143,18.188571,44122.428571
4,2010-01-14,18.618571,18.632143,18.607857,18.626457,43782.071429
...,...,...,...,...,...,...
204,2010-08-02,39.166429,39.175714,39.162857,39.167857,17596.214286
205,2010-08-03,38.770357,38.771786,38.761286,38.763929,8649.142857
206,2010-08-04,39.520714,39.530000,39.509300,39.514286,25619.785714
207,2010-08-05,39.903929,39.910714,39.895714,39.907857,4858.428571


In [137]:
weekly_df.to_csv("..\..\Data\Week\\" + data_name, index = False)